# Install Dependencies

In [ ]:
!pip install neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.0/198.0 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.18.0-py3-none-any.whl size=273862 sha256=921084d9bf45ae28b6b33d0468f52f0202f838a1b42a713d1672035399d4eddf
  Stored in directory: /root/.cache/pip/wheels/e7/e1/a0/dd7c19192f5383ff57d02a6c126cbfe4b7b2ae82f70c6994ce
Successfully built neo4j


# Connect to the Database

In [ ]:
from neo4j import GraphDatabase

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "neo4j+s://bc8b6e15.databases.neo4j.io"
AUTH = ("neo4j", "Vucu9PUNiseZqd3RqqHjOBQ2-spLkBuP1H_VYbaZo7M")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

# Query the Database

In [ ]:
# Construct the schema

sample_schema = """
CREATE (BNY:COMPANY {name: 'BNY Mellon'})
CREATE (q1_2023:EARNINGSCALL {name: 'q1_2023'})
CREATE (BNY) -[:HAS_EARNINGS] -> (q1_2023)
CREATE (q1_2023) -[:HAS_SECTION]-> (p:PARTICIPANT_SECTION {name: 'Participants'})
CREATE (q1_2023) -[:HAS_SECTION]-> (presentation:PRESENTATION_SECTION {name: 'Presentation'})
CREATE (q1_2023) -[:HAS_SECTION]-> (qa:QA_SECTION {name: "Q & A"})

CREATE (p) - [:HAS_PARTICIPANT] -> (rv:PARTICIPANT {name: "Robin Vince", title: "CEO of BNY Mellon"})
CREATE (p) - [:HAS_PARTICIPANT] -> (mm:PARTICIPANT {name: "Mike Mayo", title: "Analyst Wells Fargo"})

CREATE (presentation) -[:PRESENT_BY]-> (rv)
CREATE (presentation) -[:HAS_CONTENT]-> (content:CONTENT {name: "content", text: "The fourth quarter afsdhijbafsdhiuasdijb hiusdafihuj hiuasdijafdsihj ihu iuhjaihuj ijhdhij hiujahjkf dhij uih  asdf"})
CREATE (rv) -[:ANNOUNCE] -> (content)


CREATE (ans: ANSWER {text: "Where do you exp...", sentiment: "+ve"}) -[:ANSWER_TO] -> (question:QUESTION {text: "Where do you exp...", sentiment: "+ve"})
CREATE (mm)-[:ASKED]->(question)
CREATE (rv)-[:ANSWERED]->(ans)
CREATE (qa) -[:HAS_QUESTION]->(question)

CREATE (topic:Topic {name: "Revenue"})
CREATE (ans)-[:MENTIONED]->(topic)
CREATE (question)-[:MENTIONED]->(topic)

"""

records, summary, keys = driver.execute_query(sample_schema)

<ipython-input-8-d8d2851ba340>:30: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query(schema)


# Parsing


In [ ]:
import xml.etree.ElementTree as ET
tree = ET.parse('data.xml')
root = tree.getroot()

In [ ]:
header_element = root.find('header')

In [ ]:
class Transcript:
    def __init__(self, header, participants_section, presentation_section, qa_section):
        self.header = header
        self.participants_section = participants_section
        self.presentation_section = presentation_section
        self.qa_section = qa_section

class Header:
    def __init__(self, company, title, time, currency, note):
        self.company = company
        self.title = title
        self.time = time
        self.currency = currency
        self.note = note

class ParticipantsSection:
    def __init__(self, participants=None, name="call participants"):
        self.name = name
        self.__participantsDict__ = participants
        self.participants = participants.values()

    def get_participant(self, participants_id):
        return self.__participantsDict__[participants_id]


class Person:
    def __init__(self, id, position=None, group=None, name=None):
        self.position = position
        self.group = group
        self.id = id
        self.name = name

    def __repr__(self):
        return self.name + ' ' + self.position

class Statement: # Content in the schema
    def __init__(self, speaker:Person, text):
        self.speaker = speaker
        self.text = text


class PresentationSection:
    def __init__(self, statements=None, name="Presentation"):
        self.name = name
        self.statements = statements if statements else []


class QASection:
    def __init__(self, transitions=None, questions=None, answers=None):
        self.transitions = transitions if transitions else []
        self.__questionsDict__ = questions if questions else {}
        self.questions = questions.values()
        self.answers = answers if answers else []

    def get_question(self, question_id):
        return self.__questionsDict__[question_id]

class Transition:
    def __init__(self, speaker, text):
        self.speaker = speaker
        self.text = text

class Question:
    def __init__(self, id, speaker, text):
        self.id = id
        self.speaker = speaker
        self.text = text

class Answer:
    def __init__(self, id, question, speaker, text):
        self.id = id
        self.question=question
        self.speaker = speaker
        self.text = text


In [ ]:
def extract_participants(section_element, header):
    operator = Person(
            id="-1",
            name=f"{header.company} operator"
        )
    participants = {"-1": operator}
    for person_element in section_element.findall('person'):
        person = Person(
            id=person_element.get('id'),
            position=person_element.get('position'),
            group=person_element.get('group'),
            name=person_element.text
        )
        participants[person_element.get('id')] = person
    par_sec = ParticipantsSection(participants)
    return par_sec

def extract_presentation(section_element, participants):
    statements = []
    for statement_element in section_element.findall('statement'):
        speaker_element = statement_element.find('speaker')
        speaker_id=speaker_element.get('id')

        statement = Statement(speaker=participants.get_participant(speaker_id), text=speaker_element.find('text').text)
        statements.append(statement)
    return PresentationSection(statements)

def extract_qanda(section_element, participants):
    questions = {}
    answers = []
    transitions = []
    for question_element in section_element.findall('question'):
        speaker_element = question_element.find('speaker')
        question = Question(
            id=question_element.get('id'),
            speaker=participants.get_participant(speaker_element.get('id')),
            text=speaker_element.find('text').text
        )
        questions[question.id] = question

    for transition_element in section_element.findall('trainsition'): # This is a typo
        speaker_element = transition_element.find('speaker')
        transition = Transition(
            speaker=participants.get_participant("-1"), # Operator
            text=speaker_element.find('text').text
        )
        transitions.append(transition)

    for answer_element in section_element.findall('answer'):
        speaker_element = answer_element.find('speaker')
        answer = Answer(
            id=answer_element.get('id'),
            question=questions[answer_element.get('id')],
            speaker=participants.get_participant(speaker_element.get('id')),
            text=speaker_element.find('text').text
        )
        answers.append(answer)


    return QASection(transitions=transitions, questions=questions, answers=answers)


In [ ]:

# Extract header information
header_element = root.find('header')
header = Header(
    company=header_element.find('company').text,
    title=header_element.find('title').text,
    time=header_element.find('time').text,
    currency=header_element.find('currency').text,
    note=header_element.find('note').text
)

for section_element in root.findall('body/section'):
    if section_element.get('name') == "call participants":
        participants_section = extract_participants(section_element, header)
    elif section_element.get('name') == "Presentation ":
        presentation_section = extract_presentation(section_element, participants_section)
    elif section_element.get('name') == "Question and Answer":
        qa_section = extract_qanda(section_element, participants_section)
    else:
        print(f"Skip tag {section_element.get('name')}")

transcript = Transcript(header, participants_section, presentation_section, qa_section)

Skip tag financial tables


In [ ]:
header.title

'FQ2 2023 Earnings Call Transcripts'

In [ ]:
COMPANY = "COMPANY"
EARNINGSCALL = "EARNINGSCALL"
PARTICIPANT_SECTION = "PARTICIPANT_SECTION"
PRESENTATION_SECTION = "PRESENTATION_SECTION"
QA_SECTION = "QA_SECTION"
def add_query(cyper, query):
    return cyper + query + "\n"

def make_participant_id(id):
    return f"participant{int(id)+1}" #offset -1

def make_content_id(id):
    return f"content{id}"

def make_question_id(id):
    return f"question{id}"
def make_answer_id(id):
    return f"answer{id}"

cypher = ""
# process_header:
header = transcript.header
query = \
    'MERGE (%s:COMPANY {name: "%s"}) \n' % (COMPANY, header.company) + \
    'MERGE (%s:EARNINGSCALL {name: "%s", time: "%s"}) \n' % (EARNINGSCALL, header.title, header.time) + \
    'MERGE (%s) -[:HAS_EARNINGS] -> (%s)' % (COMPANY, EARNINGSCALL)

cypher = add_query(cypher, query)

# process_participants:
participants_section = transcript.participants_section
query = 'MERGE (%s) -[:HAS_SECTION]-> (%s:SECTION {name: "%s"}) \n' % (EARNINGSCALL, PARTICIPANT_SECTION, PARTICIPANT_SECTION)
for participant in participants_section.participants:
    q = 'MERGE (PARTICIPANT_SECTION) - [:HAS_PARTICIPANT] -> (%s:PARTICIPANT {name: "%s", position: "%s", group: "%s", id: "%s"})' \
    % (make_participant_id(participant.id), participant.name, participant.position, participant.group, participant.id)
    query = add_query(query, q)

cypher = add_query(cypher, query)


# process_presentation:
presentation_section = transcript.presentation_section
query = 'MERGE (%s) -[:HAS_SECTION]-> (%s:SECTION {name: "%s"}) \n' % (EARNINGSCALL, PRESENTATION_SECTION, PRESENTATION_SECTION)
content_id = 0
for content in presentation_section.statements:
    q = 'MERGE (%s) -[:HAS_CONTENT]-> (%s:CONTENT {text: "%s"}) \n' % (PRESENTATION_SECTION, make_content_id(content_id), content.text) + \
        'MERGE (%s) -[:ANNOUNCE] -> (%s) \n' % (make_participant_id(content.speaker.id), make_content_id(content_id))
    content_id += 1
    query = add_query(query, q)

cypher = add_query(cypher, query)

# process_qa_section:
# Questions
qa_section = transcript.qa_section
query = 'MERGE (%s) -[:HAS_SECTION]-> (%s:SECTION {name: "%s"}) \n' % (EARNINGSCALL, QA_SECTION, QA_SECTION)
for question in qa_section.questions:
    # TODO: add sentiment analysis!
    q = 'MERGE (%s) -[:HAS_QUESTION]->(%s:QUESTION {text: "%s", sentiment: ""}) \n' % (QA_SECTION, make_question_id(question.id), question.text) + \
        'MERGE (%s)-[:ASKED]->(%s)' % (make_participant_id(question.speaker.id), make_question_id(question.id))
    # TODO: add topic modeling and associate question to topics
    query = add_query(query, q)
cypher = add_query(cypher, query)

# Answer
query = ""
answer_id = 0
for answer in qa_section.answers:
    # TODO: add sentiment analysis!
    q = 'MERGE (%s: ANSWER {text: "%s", sentiment: ""}) -[:ANSWER_TO] -> (%s) \n' % (make_answer_id(answer_id), answer.text, make_question_id(answer.question.id)) + \
        'MERGE (%s)-[:ANSWERED]->(%s) \n' % (make_participant_id(answer.speaker.id), make_answer_id(answer_id)) + \
        'MERGE (%s) -[:HAS_ANSWER] -> (%s)' % (make_question_id(answer.question.id), make_answer_id(answer_id)) + \
        'MERGE (%s)-[:WAS_ANSWERED_BY]->(%s) \n' % (make_answer_id(answer_id), make_participant_id(answer.speaker.id))
    query = add_query(query, q)
    answer_id += 1
cypher = add_query(cypher, query)





In [ ]:
records, summary, keys = driver.execute_query(cypher)

<ipython-input-83-81b8fd8e289d>:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query(cypher)


In [ ]:
def addParticipantsSection()

'Transcript'

In [ ]:
# Get the name of all 42 year-olds
records, summary, keys = driver.execute_query(
    """MATCH (n:Product)<-[r:CONTAINS]-(s:Order)
    RETURN n,r,s
    LIMIT 25""",
    age=42,
    database_="neo4j",
)

# Loop through results and do something with them
for r in records:
    print(r['n']['productName'], r['s']['orderDate'])

# Summary information
print("The query `{query}` returned {records_count} records in {time} ms.".format(
    query=summary.query, records_count=len(records),
    time=summary.result_available_after,
))

<ipython-input-4-c5bcbfafdb46>:2: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query(


The query `MATCH (n:Product)<-[r:CONTAINS]-(s:Order)
    RETURN n,r,s
    LIMIT 25` returned 0 records in 31 ms.


# Close the Database Connection

In [ ]:
driver.close()